## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [34]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(shape=[28 * 28, 256], dtype=tf.float32, 
            initial_value=tf.random.uniform(shape=[28 * 28, 256], minval=-1, maxval=1))
        self.b1 = tf.Variable(shape=[256], dtype=tf.float32, initial_value=tf.zeros(shape=[256]))

        self.W2 = tf.Variable(shape=[256, 10], dtype=tf.float32, 
            initial_value=tf.random.uniform(shape=[256, 10], minval=-1, maxval=1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(shape=[10]))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [x.shape[0], -1])
        logits = tf.matmul(tf.matmul(x, self.W1) + self.b1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [35]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [36]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 75.36055 ; accuracy 0.09055
epoch 1 : loss 64.98107 ; accuracy 0.104483336
epoch 2 : loss 59.440815 ; accuracy 0.118483335
epoch 3 : loss 55.158707 ; accuracy 0.13143334
epoch 4 : loss 51.60824 ; accuracy 0.14291666
epoch 5 : loss 48.576275 ; accuracy 0.15325
epoch 6 : loss 45.930542 ; accuracy 0.16386667
epoch 7 : loss 43.552586 ; accuracy 0.17425
epoch 8 : loss 41.365643 ; accuracy 0.18328333
epoch 9 : loss 39.328133 ; accuracy 0.19361667
epoch 10 : loss 37.423466 ; accuracy 0.2028
epoch 11 : loss 35.646027 ; accuracy 0.21228333
epoch 12 : loss 33.99663 ; accuracy 0.22201666
epoch 13 : loss 32.470554 ; accuracy 0.23285
epoch 14 : loss 31.063696 ; accuracy 0.24441667
epoch 15 : loss 29.77233 ; accuracy 0.25661665
epoch 16 : loss 28.58848 ; accuracy 0.26926666
epoch 17 : loss 27.503826 ; accuracy 0.28351668
epoch 18 : loss 26.507408 ; accuracy 0.29625
epoch 19 : loss 25.58791 ; accuracy 0.30965
epoch 20 : loss 24.736729 ; accuracy 0.32193333
epoch 21 : loss 23.946419 ; a